# SQLAlchemy-MutableSoup Examples

## Setup

In [1]:
from sqlalchemy_mutablesoup import MutableSoupType

from bs4 import BeautifulSoup
from sqlalchemy import Column, Integer, create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker, scoped_session

Standard session creation.

In [2]:
engine = create_engine('sqlite:///:memory:')
session_factory = sessionmaker(bind=engine)
Session = scoped_session(session_factory)
session = Session()
Base = declarative_base()

Define and instantiate a database model with a `MutableSoupType` column.

In [3]:
class Model(Base):
    __tablename__ = 'model'
    id = Column(Integer, primary_key=True)
    soup = Column(MutableSoupType)
    
Base.metadata.create_all(engine)

model = Model()
session.add(model)

## Example 1: Setting a `MutableSoupType` object

`MutableSoupType` objects can be set using strings or `bs4.BeautifulSoup` objects.

In [4]:
model.soup = '<p>Hello World.</p>'
model.soup

<p>Hello World.</p>

In [5]:
model.soup = BeautifulSoup('<p>Goodbye Moon.</p>', 'html.parser')
model.soup

<p>Goodbye Moon.</p>

## Example 2: Setting a soup element

Basic use.

In [6]:
model.soup.set_element(parent_selector='p', val='Hello Moon.')
session.commit()
model.soup

<p>Hello Moon.</p>

Use with `get_target` method.

In [7]:
# with `gen_target` method
def gen_span_tag(*args, **kwargs):
    print('My args are:', args)
    print('My kwargs are:', kwargs)
    return '<span></span>'

model.soup.set_element(
    parent_selector='p',
    val='Span text',
    target_selector='span',
    gen_target=gen_span_tag,
    args=['hello world'],
    kwargs={'hello': 'moon'},
)
session.commit()
print(model.soup)

My args are: ('hello world',)
My kwargs are: {'hello': 'moon'}
<p>Hello Moon.<span>Span text</span></p>


## Example 3: Using the `changed` method

In [8]:
# this change does not survive session commits
model.soup.select_one('p')['style'] = 'color:red;'
session.commit()
model.soup

<p>Hello Moon.<span>Span text</span></p>

In [9]:
# this change survives session commits
model.soup.select_one('p')['style'] = 'color:red;'
model.soup.changed()
session.commit()
model.soup

<p style="color:red;">Hello Moon.<span>Span text</span></p>

In [10]:
model.soup.get_str('p')

'<p style="color:red;">Hello Moon.<span>Span text</span></p>'

In [11]:
model.soup.set_element('p', '<p>Hello Moon.</p>')
model.soup

<p style="color:red;"><p>Hello Moon.</p></p>